## Tokenization

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences = [
    "Eu amo meu doguinho",
    "Eu amo pipoquinha"
]

In [ ]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'eu': 1, 'amo': 2, 'meu': 3, 'doguinho': 4, 'pipoquinha': 5}


## Sequencing

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences = [
    "Eu amo meu doguinho",
    "Eu amo pipoquinha",
    "Você gosta de jujuba",
    "O mamão é uma das frutas mais nutritivas e populares do mundo e é consumido preferencialmente como fruta fresca."
]

In [ ]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

print(word_index)
print("\n")
print(sequences)

{'eu': 1, 'amo': 2, 'é': 3, 'e': 4, 'meu': 5, 'doguinho': 6, 'pipoquinha': 7, 'você': 8, 'gosta': 9, 'de': 10, 'jujuba': 11, 'o': 12, 'mamão': 13, 'uma': 14, 'das': 15, 'frutas': 16, 'mais': 17, 'nutritivas': 18, 'populares': 19, 'do': 20, 'mundo': 21, 'consumido': 22, 'preferencialmente': 23, 'como': 24, 'fruta': 25, 'fresca': 26}


[[1, 2, 5, 6], [1, 2, 7], [8, 9, 10, 11], [12, 13, 3, 14, 15, 16, 17, 18, 4, 19, 20, 21, 4, 3, 22, 23, 24, 25, 26]]


#### token oov

In [ ]:
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

test = [
    "O buriti é um fruto de alto valor nutricional",
    "O barbatimão é conhecido por suas diversas propriedades medicinais"
]

test_seq = tokenizer.texts_to_sequences(test)

print(test_seq)

[[13, 1, 4, 1, 1, 11, 1, 1, 1], [13, 1, 4, 1, 1, 1, 1, 1, 1]]


## pad_sequences

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

test = [
    "O buriti é um fruto conhecido por seu alto valor nutricional",
    "O barbatimão é conhecido por suas propriedades medicinais",
    "O jamelão é roxinho"
]

sequences = tokenizer.texts_to_sequences(test)

padded = pad_sequences(sequences, padding="post")

print(padded[0])
print(padded.shape)

[13  1  4  1  1  1  1  1  1  1  1]
(3, 11)


#### treinando um modelo

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

In [ ]:
sentences = [
    "O buriti é um fruto conhecido por seu alto valor nutricional",
    "O barbatimão é conhecido por suas propriedades medicinais",
    "O jamelão é roxinho"
]

labels = [1, 1, 0]  # Exemplo: 1 para "nutricional/medicinal", 0 para outroslabels = [0, 1, 0]  # Substitua pelos seus rótulos (ex.: 0 ou 1 para classificação binária)

vocab_size = 100  # Tamanho máximo do vocabulário
embedding_dim = 16  # Dimensão dos vetores de embedding
max_length = 10    # Comprimento máximo das sequências
padding_type = "post"  # Tipo de padding (pós ou pré)
trunc_type = "post"    # Tipo de truncamento (pós ou pré)

In [ ]:
# Divisão dos dados em conjuntos de treinamento e teste
training_size = 2  # Define o tamanho do conjunto de treinamento (2 frases)
training_sentences = sentences[0:training_size]  # Seleciona as primeiras 'training_size' frases
testing_sentences = sentences[training_size:]   # Seleciona as frases restantes para teste
training_labels = labels[0:training_size]       # Seleciona os rótulos correspondentes ao treinamento
testing_labels = labels[training_size:]         # Seleciona os rótulos correspondentes ao teste
training_labels_np = np.array(training_labels)
testing_labels_np = np.array(testing_labels)

In [ ]:
# Configuração do tokenizador
tokenizer = Tokenizer(num_words=vocab_size, oov_token="oov_tok")  # Cria o tokenizador com tamanho de vocabulário e token OOV
tokenizer.fit_on_texts(training_sentences)  # Constrói o vocabulário com base nas frases de treinamento
word_index = tokenizer.word_index  # Armazena o mapeamento de palavras para índices


# Conversão das frases em sequências numéricas
training_sequences = tokenizer.texts_to_sequences(training_sentences)  # Converte frases de treinamento em sequências
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)  # Padroniza o comprimento das sequências
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)  # Converte frases de teste em sequências
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)  # Padroniza o comprimento das sequências

In [ ]:
# Definição do modelo
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),  # Camada de embedding para representar palavras como vetores
    GlobalAveragePooling1D(),  # Camada de pooling para reduzir a dimensionalidade
    Dense(24, activation='relu'),  # Camada densa com ativação ReLU
    Dense(1, activation='sigmoid')  # Camada de saída com ativação sigmoid para classificação binária
])

# Compilação do modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Configura a função de perda, otimizador e métricas

In [ ]:
# Treinamento do modelo (assumindo que training_padded e training_labels estão prontos)
history = model.fit(training_padded, training_labels_np, epochs=10, validation_data=(testing_padded, testing_labels_np))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.6969 - val_accuracy: 0.0000e+00 - val_loss: 0.7006
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.5000 - loss: 0.6934 - val_accuracy: 0.0000e+00 - val_loss: 0.7042
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5000 - loss: 0.6902 - val_accuracy: 0.0000e+00 - val_loss: 0.7078
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 1.0000 - loss: 0.6868 - val_accuracy: 0.0000e+00 - val_loss: 0.7115
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 1.0000 - loss: 0.6832 - val_accuracy: 0.0000e+00 - val_loss: 0.7152
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 1.0000 - loss: 0.6796 - val_accuracy: 0.0000e+00 - val_loss: 0.7189
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 1.0000 - loss: 0.6762 - val_accuracy: 0.0000e+00 - val_loss: 0.7227
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.6730 - val_a

In [ ]:
loss, accuracy = model.evaluate(testing_padded, testing_labels_np)
print(f"Loss: {loss}, Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0000e+00 - loss: 0.7342
Loss: 0.734200119972229, Accuracy: 0.0
